In [ ]:
# Lets build an RNN

import torch
import torch.nn as nn
import torch.optim as optim

class SimpleRNN(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(SimpleRNN,self).__init__()

    # self.fc1 = nn.Linear(input_size,hidden_size)
    # self.relu = nn.ReLU()
    self.rnn = nn.RNN(input_size,hidden_size,num_layers=2)
    self.fc2 = nn.Linear(hidden_size,output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    out, _ = self.rnn(x)
    return self.sigmoid(self.fc2(out[:, -1]))

In [ ]:

# --- 1. DATA DIMENSIONS SETUP ---
seq_len = 10    # Look back at the last 10 days
features = 5    # OHLCV (Open, High, Low, Close, Volume)

# Create dummy data: 32 samples, each is a 10-day window, 5 stats per day
X = torch.randn(100, seq_len, features)
# Target: 32 labels (e.g., price goes up=1 or down=0)
Y = torch.randint(0, 2, (100, 1)).float()

model = SimpleRNN(5,45,1)
optimizer = optim.AdamW(model.parameters(),lr = 0.01)
scheduler= optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',factor=0.5,patience=15)

criterion = nn.BCELoss()

for epoch in range(2000):

  outputs = model(X)
  loss = criterion(outputs,Y)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  scheduler.step(loss)


  if epoch % 20 == 0:
    curr_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch [{epoch}/100], Loss: {loss.detach().item():.4f}, LR: {curr_lr}")
